# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [31]:
import os
import json
import requests
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv




In [32]:

# --- Load environment ---
load_dotenv()

MODEL_LLAMA = os.getenv("LOCAL_MODEL_NAME", "llama3.2")
OLLAMA_BASE = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434/v1")

In [33]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""


In [34]:


print(f"Getting explanation from {MODEL_LLAMA} using Ollama...")

try:
    response = requests.post(
        f"{OLLAMA_BASE}/completions",
        json={
            "model": MODEL_LLAMA,
            "prompt": question,
            "stream": True
        },
        stream=True,
        timeout=120
    )

    output = ""
    display_handle = display(Markdown("Generating response..."), display_id=True)

    for line in response.iter_lines(decode_unicode=True):
        if not line.strip():
            continue

        # Each event line starts with "data: "
        if line.startswith("data: "):
            line = line[len("data: "):]

        if line.strip() == "[DONE]":
            break

        try:
            data = json.loads(line)
        except json.JSONDecodeError:
            continue

        # In Ollama /v1/completions, the text comes in data['choices'][0]['text']
        text = data.get("choices", [{}])[0].get("text", "")
        if text:
            output += text
            update_display(Markdown(output), display_id=display_handle.display_id)

    print("\nFinal explanation:\n")
    display(Markdown(f"### Llama 3.2 Explanation\n\n{output.strip()}"))

except requests.exceptions.ConnectionError:
    print("Could not connect to Ollama — make sure it’s running (run `ollama serve`).")
except Exception as e:
    print("Unexpected error:", e)


Getting explanation from llama3.2 using Ollama...


This line of code is using the `yield from` keyword in Python, which is used for iterating over multiple iterators. Let's break it down:

```python
for book in books:
```

This part starts a loop that will iterate over an iterable called `books`. This could be any type of object, such as a list, set, dictionary iterator, etc.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

 Inside this loop, another generator expression is using `yield from` to send the results of several sub-generators through its parent generator. In this case:

1. `(book.get("author"))`: This is a generator expression that generates values by applying a function (`get`) to each item in an iterable called `book`. It assumes that each book has a key called `"author"` containing some author name.

2. `{... for book in books if book.get("author")}`: Before the generator can run, there's a condition that needs to be met for it to start its sub-generations: that book indeed contains an `"author"`. That's exactly what `if book.get("author")` does - checks whether or not the sub-expression is true before starting each sub-generation.

These two together means you are only generating values from books, where `book.hasAuthor()` function results in True. In Python 3.10 and later, we don't need to check if this value exists before trying to get it out of the dictionary with "author" key. In earlier Python versions we do that.

In summary, when you look at your code as a whole, you can see two nested `for` loops - an outer loop that goes through each book, and for which there is a sub-loop where author names are produced in sequence so you could use them somewhere later.

```python
# This is the example code that uses 'yield from'
books = [
        {'id':0,"name":"Python Programming"},
        {'id' : 2, "name": "A Book on Python programming" },
        {"id"      :"4", "name":"Python Documentation"}
   ]


def get_all_authors(books):
    for book in books:
         yield book.get("author") 

# usage
all_authors = [a for a in get_all_authors(books) if a]
print(all_authors)
```

  The code does not handle cases where there is no '"author"' inside each dictionary. In 'real world applications' it might raise an exception and make more sense.


Final explanation:



### Llama 3.2 Explanation

This line of code is using the `yield from` keyword in Python, which is used for iterating over multiple iterators. Let's break it down:

```python
for book in books:
```

This part starts a loop that will iterate over an iterable called `books`. This could be any type of object, such as a list, set, dictionary iterator, etc.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

 Inside this loop, another generator expression is using `yield from` to send the results of several sub-generators through its parent generator. In this case:

1. `(book.get("author"))`: This is a generator expression that generates values by applying a function (`get`) to each item in an iterable called `book`. It assumes that each book has a key called `"author"` containing some author name.

2. `{... for book in books if book.get("author")}`: Before the generator can run, there's a condition that needs to be met for it to start its sub-generations: that book indeed contains an `"author"`. That's exactly what `if book.get("author")` does - checks whether or not the sub-expression is true before starting each sub-generation.

These two together means you are only generating values from books, where `book.hasAuthor()` function results in True. In Python 3.10 and later, we don't need to check if this value exists before trying to get it out of the dictionary with "author" key. In earlier Python versions we do that.

In summary, when you look at your code as a whole, you can see two nested `for` loops - an outer loop that goes through each book, and for which there is a sub-loop where author names are produced in sequence so you could use them somewhere later.

```python
# This is the example code that uses 'yield from'
books = [
        {'id':0,"name":"Python Programming"},
        {'id' : 2, "name": "A Book on Python programming" },
        {"id"      :"4", "name":"Python Documentation"}
   ]


def get_all_authors(books):
    for book in books:
         yield book.get("author") 

# usage
all_authors = [a for a in get_all_authors(books) if a]
print(all_authors)
```

  The code does not handle cases where there is no '"author"' inside each dictionary. In 'real world applications' it might raise an exception and make more sense.